In [4]:
import tushare as ts
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels import regression
import ffn
from sklearn.svm import SVR
import matplotlib.pyplot as plt

In [5]:
#获取数据
data1 = ts.get_sz50s()
data1.head()

,code,name
0,600000,浦发银行
1,600016,民生银行
2,600028,中国石化
3,600029,南方航空
4,600030,中信证券


In [6]:
#获取股票id
stocks_ID = data1.loc[:,'code']
stocks_ID.head()


0    600000
1    600016
2    600028
3    600029
4    600030
Name: code, dtype: object

In [7]:
#获取股票2017-3月到6月的股票数据作为训练集 将2017-7月份的数据作为测试集

flag  = 1 
for i in stocks_ID:
    i = str(i)
    if flag == 1:
        train_data = ts.get_hist_data(i,start='2017-03-01',end='2017-06-30')
        test_data = ts.get_hist_data(i,start='2017-07-01',end='2017-08-31')
        flag = 0
    else:
        tmp_data = ts.get_hist_data(i,start='2017-03-01',end='2017-06-30')
        tmp1_data = ts.get_hist_data(i,start='2017-07-01',end='2017-08-31')
        train_data.append(tmp_data)
        test_data.append(tmp1_data)
    


In [11]:
#将需要的特征提取出来
train_data = train_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
test_data = test_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]

In [54]:
#计算开盘到收盘之间的价格均值
def cal_mean(train_data,test_data):
    first_data_open = train_data.loc[:,['open']]
    first_data_close = train_data.loc[:,['close']]

    second_data_open = test_data.loc[:,['open']]
    second_data_close = test_data.loc[:,['close']]
    
    first_data_open.rename(columns = {'open':'mean'},inplace=True)
    first_data_close.rename(columns = {'close':'mean'},inplace=True)
    first_data_mean = (first_data_close + first_data_open)/2
    
    second_data_open.rename(columns = {'open':'mean'},inplace=True)
    second_data_close.rename(columns = {'close':'mean'},inplace=True)
    second_data_mean = (second_data_close + second_data_open)/2
    
    train_data['mean'] = first_data_mean
    test_data['mean'] = second_data_mean
    return train_data,test_data




In [94]:
#计算N期收益
def cal_nReturn(train_data, n):
    #训练集的N期平均收益率
    origin_mean = train_data.loc[:,['mean']]
    origin_mean_n = origin_mean.shift(n)
    train_simpleret = (origin_mean - origin_mean_n) / origin_mean_n
    train_data['n_simpleret'] = train_simpleret
    #填充缺失的值
    train_data = train_data.fillna(method='bfill')
    return train_data
    
    

In [95]:
train_data = cal_nReturn(train_data,15)
train_data.head(1)

,open,high,close,low,price_change,p_change,turnover,mean,n_simpleret
date,,,,,,,,,
2017-06-30,12.7,12.73,12.65,12.59,-0.11,-0.86,0.13,12.675,0.014596


In [108]:
def Train(train_data,test_data):
    #使用scikit-learn导入SVR进行训练和预测
    svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)
    X = train_data.iloc[:,-9:-2]
    X1 = test_data.iloc[:,-8:-1]
    y = train_data.iloc[:,-1:]
    y_rbf = svr_rbf.fit(X,y).predict(X1)
    return y_rbf

In [109]:
ans = Train(train_data,test_data)
ans

E:\Anaconda\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 0.0911548 ,  0.11399597,  0.04426155,  0.02975173,  0.00693432,
        0.01739646, -0.01277529,  0.00229085, -0.00251794,  0.00084556,
       -0.00719296,  0.01689668,  0.0069064 ,  0.03554165,  0.05408975,
        0.03561231,  0.09395888,  0.09972601,  0.04523369,  0.16594638,
        0.19141191,  0.15584899,  0.10991686,  0.26909686,  0.08168549,
        0.16011668,  0.26777435,  0.33024232,  0.03450559,  0.33820559,
        0.27847066,  0.06085007,  0.36567636,  0.09337763,  0.05668389,
        0.03410035,  0.03620058,  0.00223589,  0.02117017,  0.02943223,
        0.00959853, -0.00620927,  0.00368318,  0.02978353])